# Text 2: Latent semantic indexing
**Internet Analytics - Lab 4**

---

**Group:** *Your group letter.*

**Names:**

* *Name 1*
* *Name 2*
* *Name 3*

---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [ ]:
import pickle as pk
import numpy as np
from scipy.sparse.linalg import svds

## Exercise 4.4: Latent semantic indexing

In [ ]:
k = 300
td_matrix = np.load("occ_matrix.npy")
#SVD decomposition
u, s, v = np.linalg.svd(td_matrix, full_matrices=False)


In [ ]:
u_k = u[:,:k]
v_k = v[:k,:]
s_k = s[:k]
print("20 biggest eigenvalues : \n",s[:19])

In the matrix U, each __row__ correspond to a term, and for each term the value index i correspond to how much relevant the ith concept is for this term.
In the matrix V, each __column__ correspond to a document, and for each document the value at index i correspond to how much relevant the ith concept is for this term.

In [ ]:
n = 10

wordIndex = pk.load(open("indexToWord"))
n_topic_relevant_document_index = np.argpartition(v_k, -n,1)[:,-n:]
n_topic_relevant_term_index = np.argpartition(u_k, n,0)[-n:]
n_topic_relevant_document_index.shape

for i in range(n):
    print("*************************************************")
    print("In topic",i,"the relevant terms are:")
    print(",".join(map(wordIndex.get,n_topic_relevant_term_index[i])))


## Exercise 4.5: Topic extraction

## Exercise 4.6: Document similarity search in concept-space

## Exercise 4.7: Document-document similarity